In [ ]:
import http.client
import pandas as pd
import json


In [ ]:
# getting first page
conn = http.client.HTTPSConnection("api.twitch.tv")
payload = ''
with open('tokens')as f:
    client_id = f.readline().strip();
    authorization = f.readline().strip();
    
headers = {
    'Client-Id': client_id,
    'Authorization': f'Bearer {authorization}',
}


In [ ]:
conn.request("GET", "/helix/streams?first=100", payload, headers)
res = conn.getresponse()
data = res.read()
json_ = data.decode("utf-8")
json_ = json.loads(json_)

In [ ]:
data_list = json_['data']
pagination = json_['pagination']['cursor']

In [ ]:
# getting 100 pages
for i in range(100):
    conn.request("GET", f"/helix/streams?first=100&after={pagination}", payload, headers)
    res = conn.getresponse()
    data = res.read()
    json_ = data.decode("utf-8")
    json_ = json.loads(json_)
    data_list.extend(json_['data'])
    pagination = json_['pagination']['cursor']

In [ ]:
# saving json list of data
with open('data/json_list.json', 'w') as f:
    json.dump(data_list, f)

In [ ]:
# loading json list of data
with open('data/json_list.json') as f:
    data_list = json.load(f)


In [ ]:
# poping tag ids which are nested, so that ruins df
[i.pop('tag_ids') for i in data_list]


In [19]:

# saving dataframe
df = pd.DataFrame.from_records(data_list)
df.to_csv('data/sraped_data.csv')